In [1]:
from pprint import pprint 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import praw
user_agent="Scraper 1.0 by /u/kav_yay"
reddit=praw.Reddit(
client_id="YCp6d62ye13q-A",
client_secret="ytcu1vnZJyHO_pK2PZC_fX2Q6ImfSA",
user_agent=user_agent
)

In [3]:
title_list = []
num_comments_list = []
link_flair_text_list = []
body_list = []

In [4]:
flairs=["TW Multiple Trigger Warning","TW Physical Violence","Preparing To Leave","TW: Threats of Violence"]

In [5]:
subreddit = reddit.subreddit('domesticviolence')
hot_post = subreddit.hot(limit=None)
for sub in hot_post:
    if sub.link_flair_text=="TW Multiple Trigger Warning" or sub.link_flair_text=="TW Physical Violence" or sub.link_flair_text=="Preparing To Leave" or sub.link_flair_text=="TW: Threats of Violence":
        title_list.append(sub.title)
        link_flair_text_list.append(sub.link_flair_text)
        num_comments_list.append(sub.num_comments)
        body_list.append(sub.selftext)

In [6]:
df = pd.DataFrame({ 'Title':title_list,
                   'Count_of_Comments':num_comments_list,
                   'Body':body_list,
                   'Flair':link_flair_text_list
                  })
df.to_csv('DV2.csv', index = False)


In [7]:
len(df)

429